In [801]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

In [802]:
x1 =  pd.read_csv('C:/Users/FRANK/Desktop/Proyectos Linkedin/Python Proyectos/Dia 16/train.csv') # Trabajar con este dataset
x2 =  pd.read_csv('C:/Users/FRANK/Desktop/Proyectos Linkedin/Python Proyectos/Dia 16/test.csv') # Hallar dataset con train.csv
x1.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [803]:
print('Número de observaciones (filas) = {}'.format(x1.shape[0]))
print('Total de variables disponibles (columnas) = {} '.format(x1.columns.shape[0]))
print('Columnas que cuentan con un valor nulo -')
print(x1.isnull().sum())

Número de observaciones (filas) = 7613
Total de variables disponibles (columnas) = 5 
Columnas que cuentan con un valor nulo -
id             0
keyword       61
location    2533
text           0
target         0
dtype: int64


In [804]:
x1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [805]:
x1['keyword'].value_counts()

keyword
fatalities               45
deluge                   42
armageddon               42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: count, Length: 221, dtype: int64

 # Limpiando el texto de X1

In [806]:
x1['keyword'].fillna('none',inplace=True)
x1['location'].fillna('none',inplace=True)

C:\Users\FRANK\AppData\Local\Temp\ipykernel_20668\1173858714.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x1['keyword'].fillna('none',inplace=True)
C:\Users\FRANK\AppData\Local\Temp\ipykernel_20668\1173858714.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

In [807]:
x1.isnull().sum()

id          0
keyword     0
location    0
text        0
target      0
dtype: int64

In [808]:
# Función de limpieza de texto
def clean_text(text):
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Eliminar menciones de usuarios
    text = re.sub(r'@\w+', '', text)
    # Eliminar hashtags
    text = re.sub(r'#', '', text)
    # Eliminar puntuación
    text = re.sub(r'[^\w\s]', '', text)
    # Convertir a minúsculas
    text = text.lower()
    # Tokenizar el texto (dividir en palabras)
    tokens = text.split()
    # Unir las palabras de nuevo en una sola cadena
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Aplicar la función de limpieza al texto
x1['cleaned_text'] = x1['text'].apply(clean_text)

In [809]:
x1['text'].unique()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'Suicide bomber kills 15 in Saudi security site mosque - Reuters via World - Google News - Wall ... http://t.co/nF4IculOje',
       'Two giant cranes holding a bridge collapse into nearby homes http://t.co/STfMbbZFB5',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [810]:
x1['cleaned_text'].unique()

array(['our deeds are the reason of this earthquake may allah forgive us all',
       'forest fire near la ronge sask canada',
       'all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected',
       ...,
       'officials say a quarantine is in place at an alabama home over a possible ebola case after developing symptoms',
       'on the flip side im at walmart and there is a bomb and everyone had to evacuate so stay tuned if i blow up or not',
       'suicide bomber kills 15 in saudi security site mosque reuters via world google news wall'],
      dtype=object)

In [811]:
# Función de limpieza de texto
def clean_text_location(location):
    # Eliminar URLs
    location = re.sub(r'http\S+|www\S+|https\S+', '', location, flags=re.MULTILINE)
    # Eliminar menciones de usuarios
    location = re.sub(r'@\w+', '', location)
    # Eliminar hashtags
    location = re.sub(r'#', '', location)
    # Eliminar puntuación
    location = re.sub(r'[^\w\s]', '', location)
    # Convertir a minúsculas
    location = location.lower()
    # Tokenizar el texto (dividir en palabras)
    tokens = location.split()
    # Unir las palabras de nuevo en una sola cadena
    cleaned_location = ' '.join(tokens)
    return cleaned_location

# Aplicar la función de limpieza al texto
x1['cleaned_location'] = x1['location'].apply(clean_text_location)

In [812]:
x1['location'].unique()

array(['none', 'Birmingham', 'Est. September 2012 - Bristol', ...,
       'Vancouver, Canada', 'London ', 'Lincoln'], dtype=object)

In [813]:
x1['cleaned_location'].unique()

array(['none', 'birmingham', 'est september 2012 bristol', ...,
       'denton texas', 'newcastleupontyne uk', 'lincoln'], dtype=object)

In [814]:
# Función de limpieza de palabras clave
def clean_text_keyword(keyword):
    # Eliminar URLs
    keyword = re.sub(r'http\S+|www\S+|https\S+', '', keyword, flags=re.MULTILINE)
    # Eliminar menciones de usuarios
    keyword = re.sub(r'@\w+', '', keyword)
    # Eliminar hashtags
    keyword = re.sub(r'#', '', keyword)
    # Eliminar puntuación
    keyword = re.sub(r'[^\w\s]', '', keyword)
    # Convertir a minúsculas
    keyword = keyword.lower()
    # Eliminar palabras seguidas de números
    keyword = re.sub(r'\b\w*\d\w*\b', '', keyword)
    # Tokenizar el texto (dividir en palabras)
    tokens = keyword.split()
    # Unir las palabras de nuevo en una sola cadena
    cleaned_keyword = ' '.join(tokens)
    return cleaned_keyword

# Aplicar la función de limpieza a las palabras clave
x1['cleaned_keyword'] = x1['keyword'].apply(clean_text_keyword)

In [815]:
x1['keyword'].unique()

array(['none', 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', '

In [816]:
x1['cleaned_keyword'].unique()

array(['none', 'ablaze', 'accident', 'aftershock', '', 'ambulance',
       'annihilated', 'annihilation', 'apocalypse', 'armageddon', 'army',
       'arson', 'arsonist', 'attack', 'attacked', 'avalanche', 'battle',
       'bioterror', 'bioterrorism', 'blaze', 'blazing', 'bleeding',
       'blight', 'blizzard', 'blood', 'bloody', 'bomb', 'bombed',
       'bombing', 'burned', 'burning', 'casualties', 'casualty',
       'catastrophe', 'catastrophic', 'collapse', 'collapsed', 'collide',
       'collided', 'collision', 'crash', 'crashed', 'crush', 'crushed',
       'curfew', 'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths',
       'debris', 'deluge', 'deluged', 'demolish', 'demolished',
       'demolition', 'derail', 'derailed', 'derailment', 'desolate',
       'desolation', 'destroy', 'destroyed', 'destruction', 'detonate',
       'detonation', 'devastated', 'devastation', 'disaster', 'displaced',
       'drought', 'drown', 'drowned', 'drowning', 'earthquake',
       'electrocute',

In [817]:
x1.head()

,id,keyword,location,text,target,cleaned_text,cleaned_location,cleaned_keyword
0,1,none,none,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,none,none
1,4,none,none,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,none,none
2,5,none,none,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,none,none
3,6,none,none,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,none,none
4,7,none,none,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,none,none


# Limpiando el texto x2 

In [818]:
x2.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [819]:
print('Número de observaciones (filas) = {}'.format(x2.shape[0]))
print('Total de variables disponibles (columnas) = {} '.format(x2.columns.shape[0]))
print('Columnas que cuentan con un valor nulo -')
print(x2.isnull().sum())

Número de observaciones (filas) = 3263
Total de variables disponibles (columnas) = 4 
Columnas que cuentan con un valor nulo -
id             0
keyword       26
location    1105
text           0
dtype: int64


In [820]:
x2['keyword'].fillna('none',inplace=True)
x2['location'].fillna('none',inplace=True)

C:\Users\FRANK\AppData\Local\Temp\ipykernel_20668\2606751553.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x2['keyword'].fillna('none',inplace=True)
C:\Users\FRANK\AppData\Local\Temp\ipykernel_20668\2606751553.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

In [821]:
# Función de limpieza de texto
def clean_text(text):
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Eliminar menciones de usuarios
    text = re.sub(r'@\w+', '', text)
    # Eliminar hashtags
    text = re.sub(r'#', '', text)
    # Eliminar puntuación
    text = re.sub(r'[^\w\s]', '', text)
    # Convertir a minúsculas
    text = text.lower()
    # Tokenizar el texto (dividir en palabras)
    tokens = text.split()
    # Unir las palabras de nuevo en una sola cadena
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Aplicar la función de limpieza al texto
x2['cleaned_text'] = x2['text'].apply(clean_text)

In [822]:
x2['text'].unique()

array(['Just happened a terrible car crash',
       'Heard about #earthquake is different cities, stay safe everyone.',
       'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all',
       ..., 'Green Line derailment in Chicago http://t.co/UtbXLcBIuY',
       'MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3',
       '#CityofCalgary has activated its Municipal Emergency Plan. #yycstorm'],
      dtype=object)

In [823]:
x2['cleaned_text'].unique()

array(['just happened a terrible car crash',
       'heard about earthquake is different cities stay safe everyone',
       'there is a forest fire at spot pond geese are fleeing across the street i cannot save them all',
       ..., 'green line derailment in chicago',
       'meg issues hazardous weather outlook hwo',
       'cityofcalgary has activated its municipal emergency plan yycstorm'],
      dtype=object)

In [824]:
# Función de limpieza de texto
def clean_text_location(location):
    # Eliminar URLs
    location = re.sub(r'http\S+|www\S+|https\S+', '', location, flags=re.MULTILINE)
    # Eliminar menciones de usuarios
    location = re.sub(r'@\w+', '', location)
    # Eliminar hashtags
    location = re.sub(r'#', '', location)
    # Eliminar puntuación
    location = re.sub(r'[^\w\s]', '', location)
    # Convertir a minúsculas
    location = location.lower()
    # Tokenizar el texto (dividir en palabras)
    tokens = location.split()
    # Unir las palabras de nuevo en una sola cadena
    cleaned_location = ' '.join(tokens)
    return cleaned_location

# Aplicar la función de limpieza al texto
x2['cleaned_location'] = x2['location'].apply(clean_text_location)

In [825]:
x2['location'].unique()

array(['none', 'London', "Niall's place | SAF 12 SQUAD |", ...,
       'Acey mountain islanddåÇTorontoåÈ', 'los angeles',
       'Brussels, Belgium'], dtype=object)

In [826]:
x2['cleaned_location'].unique()

array(['none', 'london', 'nialls place saf 12 squad', ...,
       'seattle washington', 'acey mountain islanddåçtorontoåè',
       'brussels belgium'], dtype=object)

In [827]:
# Función de limpieza de palabras clave
def clean_text_keyword(keyword):
    # Eliminar URLs
    keyword = re.sub(r'http\S+|www\S+|https\S+', '', keyword, flags=re.MULTILINE)
    # Eliminar menciones de usuarios
    keyword = re.sub(r'@\w+', '', keyword)
    # Eliminar hashtags
    keyword = re.sub(r'#', '', keyword)
    # Eliminar puntuación
    keyword = re.sub(r'[^\w\s]', '', keyword)
    # Convertir a minúsculas
    keyword = keyword.lower()
    # Eliminar palabras seguidas de números
    keyword = re.sub(r'\b\w*\d\w*\b', '', keyword)
    # Tokenizar el texto (dividir en palabras)
    tokens = keyword.split()
    # Unir las palabras de nuevo en una sola cadena
    cleaned_keyword = ' '.join(tokens)
    return cleaned_keyword

# Aplicar la función de limpieza a las palabras clave
x2['cleaned_keyword'] = x2['keyword'].apply(clean_text_keyword)

In [828]:
x2['keyword'].unique()

array(['none', 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', '

In [829]:
x2['cleaned_keyword'].unique()

array(['none', 'ablaze', 'accident', 'aftershock', '', 'ambulance',
       'annihilated', 'annihilation', 'apocalypse', 'armageddon', 'army',
       'arson', 'arsonist', 'attack', 'attacked', 'avalanche', 'battle',
       'bioterror', 'bioterrorism', 'blaze', 'blazing', 'bleeding',
       'blight', 'blizzard', 'blood', 'bloody', 'bomb', 'bombed',
       'bombing', 'burned', 'burning', 'casualties', 'casualty',
       'catastrophe', 'catastrophic', 'collapse', 'collapsed', 'collide',
       'collided', 'collision', 'crash', 'crashed', 'crush', 'crushed',
       'curfew', 'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths',
       'debris', 'deluge', 'deluged', 'demolish', 'demolished',
       'demolition', 'derail', 'derailed', 'derailment', 'desolate',
       'desolation', 'destroy', 'destroyed', 'destruction', 'detonate',
       'detonation', 'devastated', 'devastation', 'disaster', 'displaced',
       'drought', 'drown', 'drowned', 'drowning', 'earthquake',
       'electrocute',

In [830]:
x2.tail()

,id,keyword,location,text,cleaned_text,cleaned_location,cleaned_keyword
3258,10861,none,none,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,earthquake safety los angeles ûò safety fasten...,none,none
3259,10865,none,none,Storm in RI worse than last hurricane. My city...,storm in ri worse than last hurricane my citya...,none,none
3260,10868,none,none,Green Line derailment in Chicago http://t.co/U...,green line derailment in chicago,none,none
3261,10874,none,none,MEG issues Hazardous Weather Outlook (HWO) htt...,meg issues hazardous weather outlook hwo,none,none
3262,10875,none,none,#CityofCalgary has activated its Municipal Eme...,cityofcalgary has activated its municipal emer...,none,none


# Eliminando columnas sin limpiar

In [831]:
x1.drop(columns='keyword',inplace=True)
x1.drop(columns='location',inplace=True)
x1.drop(columns='text',inplace=True)

In [832]:
x2.drop(columns='keyword',inplace=True)
x2.drop(columns='location',inplace=True)
x2.drop(columns='text',inplace=True)

In [833]:
x1.head()

,id,target,cleaned_text,cleaned_location,cleaned_keyword
0,1,1,our deeds are the reason of this earthquake ma...,none,none
1,4,1,forest fire near la ronge sask canada,none,none
2,5,1,all residents asked to shelter in place are be...,none,none
3,6,1,13000 people receive wildfires evacuation orde...,none,none
4,7,1,just got sent this photo from ruby alaska as s...,none,none


In [834]:
x2.head()

,id,cleaned_text,cleaned_location,cleaned_keyword
0,0,just happened a terrible car crash,none,none
1,2,heard about earthquake is different cities sta...,none,none
2,3,there is a forest fire at spot pond geese are ...,none,none
3,9,apocalypse lighting spokane wildfires,none,none
4,11,typhoon soudelor kills 28 in china and taiwan,none,none


In [835]:
from sklearn.feature_extraction.text import CountVectorizer

In [838]:
# Crear una instancia de CountVectorizer
vectorizer = CountVectorizer()
# Unir los textos limpios de ambos datasets para el Bag of Words
all_texts = x1['cleaned_text']
# Ajustar el vectorizador a los textos y transformarlos en una matriz de características
x1 = vectorizer.fit_transform(all_texts)

print(x1)

  (0, 10140)	1
  (0, 3987)	1
  (0, 1350)	1
  (0, 13900)	1
  (0, 11399)	1
  (0, 9903)	1
  (0, 13973)	1
  (0, 4692)	1
  (0, 8816)	1
  (0, 1060)	1
  (0, 5716)	1
  (0, 14722)	1
  (0, 1059)	1
  (1, 5707)	1
  (1, 5522)	1
  (1, 9530)	1
  (1, 8018)	1
  (1, 11904)	1
  (1, 12146)	1
  (1, 2648)	1
  (2, 1350)	2
  (2, 1059)	1
  (2, 11657)	1
  (2, 1457)	1
  (2, 14115)	1
  :	:
  (7611, 10734)	1
  (7611, 15376)	1
  (7611, 11773)	1
  (7611, 8350)	1
  (7611, 12391)	1
  (7611, 926)	1
  (7611, 7434)	1
  (7611, 14002)	1
  (7611, 3253)	1
  (7611, 4714)	2
  (7611, 10785)	1
  (7611, 13450)	1
  (7611, 9707)	1
  (7611, 7307)	1
  (7612, 13900)	1
  (7612, 2549)	1
  (7612, 2608)	1
  (7612, 9252)	1
  (7612, 9592)	1
  (7612, 8103)	1
  (7612, 9735)	1
  (7612, 15312)	1
  (7612, 6844)	1
  (7612, 742)	1
  (7612, 11344)	1


In [839]:
# Crear una instancia de CountVectorizer
vectorizer = CountVectorizer()
# Unir los textos limpios de ambos datasets para el Bag of Words
all_texts = x2['cleaned_text']
# Ajustar el vectorizador a los textos y transformarlos en una matriz de características
x2 = vectorizer.fit_transform(all_texts)

print(x2)

  (0, 4623)	1
  (0, 3843)	1
  (0, 8333)	1
  (0, 1632)	1
  (0, 2201)	1
  (1, 3922)	1
  (1, 420)	1
  (1, 2797)	1
  (1, 4460)	1
  (1, 2542)	1
  (1, 1865)	1
  (1, 7983)	1
  (1, 7239)	1
  (1, 3023)	1
  (2, 4460)	1
  (2, 8398)	1
  (2, 3394)	1
  (2, 3286)	1
  (2, 873)	1
  (2, 7905)	1
  (2, 6457)	1
  (2, 3565)	1
  (2, 796)	1
  (2, 3332)	1
  (2, 459)	1
  :	:
  (3259, 4170)	1
  (3259, 9441)	1
  (3259, 3856)	1
  (3259, 7070)	1
  (3259, 1869)	1
  (3259, 137)	1
  (3260, 4276)	1
  (3260, 4960)	1
  (3260, 3718)	1
  (3260, 2457)	1
  (3260, 1804)	1
  (3261, 4481)	1
  (3261, 9150)	1
  (3261, 3898)	1
  (3261, 6077)	1
  (3261, 4177)	1
  (3261, 5317)	1
  (3262, 4493)	1
  (3262, 3872)	1
  (3262, 6380)	1
  (3262, 2890)	1
  (3262, 5594)	1
  (3262, 465)	1
  (3262, 9499)	1
  (3262, 1870)	1


# 2. Regresión logistica